In [1]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

from ugs import *

In [2]:
infile = "australia/data/AUS.vtk"
outfile = "costed_aus.vtk"
mesh = meshio.read(infile)

max_distance = 75000  # this is quite a small max_distance. 
                      # A smaller value means visiting far fewer nodes, so it speeds things up a lot

In [3]:
# We need to define a way to calculate cost
def travel_cost(mesh, current, _next):
    # The travel_cost can be any function, including just the distance.
    # Here, we exagerate the elevation difference, to make changing elevation more costly
    if current == _next:
        return 0
    
    z_scaling = 100.  # 100. is a random number to pick, but has quite a big impact on the resulting paths.
    
    new_current = np.append(mesh.points[current][:2], mesh.point_data['Z'][current] * z_scaling)
    new_next    = np.append(mesh.points[_next][:2],   mesh.point_data['Z'][_next]   * z_scaling)
    
    return int(np.linalg.norm(new_current - new_next))  # return as Int, just for niceness

## Prepare for parallel execution

We want to use a parallel map function to calculate the cost for all points. We can make this easier by 'baking in' the parameters of a function we know, and leave only the `starting point` as a variable.

In [4]:
from functools import partial

# We bake in the mesh and travel_cost function into the get_from_point function
get_from_point_in_mesh = partial(get_from_point, mesh = mesh, travel_cost_function = travel_cost, max_distance = max_distance)

In [5]:
# Now we can use the get_from_point function with only a point ID as a parameter, by going via the new get_from_point_in_mesh function
# Here we do a test, and see the output format: (point, total cost of all paths to that point)
print get_from_point_in_mesh(2087)

(2087, 4382359)


## Prepare input data

We don't want to calculate the LEC of a point below sea-level, so here we find all the points above sea-level, so they can be used as starting points.

In [6]:
points_above_sealevel = np.nonzero(mesh.point_data['Z'] >= 0)[0][::-1]

In [7]:
print "Total starting points available: ", points_above_sealevel.shape[0]

Total starting points available:  138726


In [ ]:
# Output data variables
all_costs = []
mesh.point_data['cost'] = np.zeros_like(mesh.point_data['Z'])

# For parallel execution, we chunk the data up into increments (inc), so we can monitor the progress
start = 0
inc = 200
stop = inc

# Run on 11 CPUs
p = Pool(11)

while start <  points_above_sealevel.shape[0]-1:
    start_time = time.time()
    # Here we use a parallel map function to send out the chunk across the CPUs in the Pool
    costs = p.map(get_from_point_in_mesh, points_above_sealevel[start:stop])
    print "From ", start, " to ", stop, " took ", time.time() - start_time, "seconds, starting with point", points_above_sealevel[start], ". Percent complete: ", 100*(float(stop)/points_above_sealevel.shape[0])
    
    # Save the data
    all_costs.extend(costs)
    
    # Write out data progressively, so we can see progress in Paraview
    for i in all_costs:
        mesh.point_data['cost'][i[0]] = i[1]
    meshio.write(outfile, mesh)
    
    # move to the next chunk of data
    start += inc
    stop += inc
    if stop >= points_above_sealevel.shape[0]:
        stop = points_above_sealevel.shape[0]-1

## Output

Show some of the results:

In [ ]:
print all_costs[:100]